In [1]:
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten, Dropout, Activation, Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras import Sequential
from sklearn.preprocessing import LabelBinarizer
import numpy as np
import pickle
import os

In [2]:
learning_rate = 0.01 #diminui o valor do learning rate pois estamos usando Adam como otimizador

## Treinamento Rede Resnet101

In [3]:
def get_images(pasta):
    exts = ['.PNG','.JPG','.JPEG','.TIFF','.GIF','.BMP']
    tot_images = 0
    for sf in [name for name in os.listdir(pasta) if os.path.isdir(os.path.join(pasta, name))]:
        subdir = os.path.join(pasta,sf)
        tot_images = tot_images + len([name for name in os.listdir(subdir) if os.path.splitext(name)[1].upper() in exts])
    return tot_images

In [4]:
# FUNÇÃO PARA LER O NOME DAS CLASSES
def get_labels(pasta):
    return [name for name in os.listdir(pasta) if os.path.isdir(os.path.join(pasta, name))];


In [5]:
EPOCHS = 50;
BATCH_SIZE = 16; #serão gerados batches de 16 imagens (a rede vê 16 imagens de cada vez durante o treino). Esse número é para facilitar o processamento conforme a memória do computador durante o treino, colocamos 16 não travar
IMG_SIZE = (224,224,3);
optimizer_name = 'SGD'

In [6]:
trainFolder = 'C:\\Users\\LDT\\Desktop\\mestrado-unifesp\\db\\train'
valFolder = 'C:\\Users\\LDT\\Desktop\\mestrado-unifesp\\db\\val'
labels = get_labels(trainFolder)
labels = np.array(labels);

# Organiza os labels em matriz e salva, para posterior uso em classificação
lb = LabelBinarizer();
labels = lb.fit_transform(labels);
f = open('C:\\Users\\LDT\\Desktop\\mestrado-unifesp\\budioes_resnet_' + optimizer_name + '_' + str(learning_rate) + ".pickle", "wb")
f.write(pickle.dumps(lb));
f.close();

ImageDataGenerator = Carrega aos poucos as imagens em memória para fazer a leitura (para não estourar a memória)

In [7]:
#ImageDataGenerator = objeto para buscar as imagens em uma pasta
#Treino
augTrain = ImageDataGenerator(rotation_range=20, width_shift_range = 0.1, height_shift_range = 0.1, 
                              shear_range = 0.15, zoom_range = [1.0, 1.25], horizontal_flip=True, 
                              fill_mode="nearest");
#Validação
augVal = ImageDataGenerator();

Relu = 

Maxpooling = selecionar o pixel com maior valor em uma dimensão 2D de 2x2 (no exemplo abaixo, poderia ser 3x3, 4x4 etc). Nesse caso, com uma janela 2x2, o maxpooling vai reduzir o tamanho da imagem pela metade.

In [9]:
#Declaração da rede RESNET
# Pega só as camadas de convolução e retreina as de classificação

baseModel = ResNet101(include_top=False, weights="imagenet", input_tensor=Input(shape=(224, 224, 3)))

In [10]:
#Como vamos usar imagenet, não faz sentido treinar a rede novamente pois já vamos usar o modelo treinado
for layer in baseModel.layers:
    layer.trainable = False

In [11]:
#Include_Top feito "Na mão":

#É preciso criar novas camadas (headModel) pra zerar os pesos da Resnet
headModel = baseModel.output
headModel = GlobalAveragePooling2D(name="avg_pool")(headModel)
headModel = Dense(len(lb.classes_), activation="softmax", name='predictions')(headModel)
#Junta tudo num modelo só
model = Model(inputs=baseModel.input, outputs=headModel)

In [12]:
#model.summary()

In [13]:
if optimizer_name == 'Adam':
    optimizer = Adam(learning_rate=learning_rate)

if optimizer_name == 'SGD':
    optimizer = SGD(learning_rate=learning_rate)

In [14]:
#No nosso modelo, o callback irá salvar o melhor modelo entre as épocas (epochs) - função Model Checkpoint
callbacks = [
    #ReduceLROnPlateau(monitor = 'val_acc',factor=0.85, patience=10, min_lr=0.000001, verbose=1),
    ModelCheckpoint('C:\\Users\\LDT\\Desktop\\mestrado-unifesp\\exemplos\\modelo_budioes_resnet_' + optimizer_name + '_' + str(learning_rate) + 
                    '-ckpnt.model', save_best_only=True, monitor='val_accuracy', mode='max', verbose = 1)
]  

In [15]:
#Reservando espaço de memória para a rede funcionar
#Ao final de cada época, será rodado o comando "callbacks"
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

In [16]:
#Função para buscar imagens
#TREINO
trainGen = augTrain.flow_from_directory(
    trainFolder, #caminho da imagem
    class_mode="categorical", #o nome da pasta onde está a imagem será o nome da classe
    target_size=(IMG_SIZE[0], IMG_SIZE[1]), #tamanho da imagem a ser redimensionada
    color_mode="rgb", #a imagem terá 3 canais RGB
    shuffle=True, #vai embaralhar as imagens enquanto faz a leitura
    batch_size=BATCH_SIZE); #de quantas em quantas imagens será feita a leitura (tamanho do BATCH)

Found 485 images belonging to 3 classes.


In [17]:
#Função para buscar imagens
#VALIDAÇÃO
valGen = augVal.flow_from_directory(
    valFolder,
    class_mode="categorical",
    target_size=(IMG_SIZE[0], IMG_SIZE[1]),
    color_mode="rgb",
    shuffle=True,
    batch_size=BATCH_SIZE);

Found 122 images belonging to 3 classes.


In [18]:
#fit_generator vai de fato treinar a rede
trained_model = model.fit(trainGen, validation_data=valGen,
                        steps_per_epoch=get_images(trainFolder)//BATCH_SIZE,
                        validation_steps=get_images(valFolder) // BATCH_SIZE,
                        epochs = EPOCHS, callbacks=callbacks, verbose =1);
#Dividir o número de imagens pelo número de batchs para garantir que cada BATCH seja lido a cada época
#tanto no treino (steps_per_epoch) quanto na validação (validation_steps)
#verbose = dá output da rede a cada final de época


Epoch 1/50
30/30 [==============================] - 20s 373ms/step - loss: 2.0170 - accuracy: 0.4968 - val_loss: 1.2659 - val_accuracy: 0.5982

Epoch 00001: val_accuracy improved from -inf to 0.59821, saving model to C:\Users\LDT\Desktop\mestrado-unifesp\exemplos\modelo_budioes_resnet_SGD_0.01-ckpnt.model
INFO:tensorflow:Assets written to: C:\Users\LDT\Desktop\mestrado-unifesp\exemplos\modelo_budioes_resnet_SGD_0.01-ckpnt.model\assets


C:\Users\LDT\anaconda3\envs\keras-env\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/50
30/30 [==============================] - 9s 272ms/step - loss: 0.4539 - accuracy: 0.8209 - val_loss: 0.3309 - val_accuracy: 0.8393

Epoch 00002: val_accuracy improved from 0.59821 to 0.83929, saving model to C:\Users\LDT\Desktop\mestrado-unifesp\exemplos\modelo_budioes_resnet_SGD_0.01-ckpnt.model
INFO:tensorflow:Assets written to: C:\Users\LDT\Desktop\mestrado-unifesp\exemplos\modelo_budioes_resnet_SGD_0.01-ckpnt.model\assets


C:\Users\LDT\anaconda3\envs\keras-env\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 3/50
30/30 [==============================] - 9s 275ms/step - loss: 0.2641 - accuracy: 0.8998 - val_loss: 0.2698 - val_accuracy: 0.8929

Epoch 00003: val_accuracy improved from 0.83929 to 0.89286, saving model to C:\Users\LDT\Desktop\mestrado-unifesp\exemplos\modelo_budioes_resnet_SGD_0.01-ckpnt.model
INFO:tensorflow:Assets written to: C:\Users\LDT\Desktop\mestrado-unifesp\exemplos\modelo_budioes_resnet_SGD_0.01-ckpnt.model\assets


...

Epoch 9/50
30/30 [==============================] - 9s 284ms/step - loss: 0.1227 - accuracy: 0.9638 - val_loss: 0.2427 - val_accuracy: 0.9107

Epoch 00009: val_accuracy did not improve from 0.92857
Epoch 10/50
30/30 [==============================] - 9s 281ms/step - loss: 0.1227 - accuracy: 0.9552 - val_loss: 0.2209 - val_accuracy: 0.9107

Epoch 00010: val_accuracy did not improve from 0.92857
Epoch 11/50
30/30 [==============================] - 9s 283ms/step - loss: 0.1086 - accuracy: 0.9680 - val_loss: 0.1900 - val_accuracy: 0.9107

Epoch 00011: val_accuracy did not improve from 0.92857
Epoch 12/50
30/30 [==============================] - 9s 281ms/step - loss: 0.0881 - accuracy: 0.9744 - val_loss: 0.1800 - val_accuracy: 0.9286

Epoch 00012: val_accuracy did not improve from 0.92857
Epoch 13/50
30/30 [==============================] - 9s 288ms/step - loss: 0.0700 - accuracy: 0.9875 - val_loss: 0.2343 - val_accuracy: 0.9107

Epoch 00013: val_accuracy did not improve from 0.92857
Epoch 

C:\Users\LDT\anaconda3\envs\keras-env\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 19/50
30/30 [==============================] - 9s 282ms/step - loss: 0.0744 - accuracy: 0.9829 - val_loss: 0.2137 - val_accuracy: 0.9286

Epoch 00019: val_accuracy did not improve from 0.94643
Epoch 20/50
30/30 [==============================] - 9s 291ms/step - loss: 0.0570 - accuracy: 0.9893 - val_loss: 0.2244 - val_accuracy: 0.9464

Epoch 00020: val_accuracy did not improve from 0.94643
Epoch 21/50
30/30 [==============================] - 9s 290ms/step - loss: 0.0525 - accuracy: 0.9936 - val_loss: 0.2233 - val_accuracy: 0.9196

Epoch 00021: val_accuracy did not improve from 0.94643
Epoch 22/50
30/30 [==============================] - 9s 291ms/step - loss: 0.0607 - accuracy: 0.9915 - val_loss: 0.2088 - val_accuracy: 0.9107

Epoch 00022: val_accuracy did not improve from 0.94643
Epoch 23/50
30/30 [==============================] - 9s 288ms/step - loss: 0.0564 - accuracy: 0.9893 - val_loss: 0.2053 - val_accuracy: 0.9107

Epoch 00023: val_accuracy did not improve from 0.94643
Epoch

C:\Users\LDT\anaconda3\envs\keras-env\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 29/50
30/30 [==============================] - 9s 286ms/step - loss: 0.0550 - accuracy: 0.9808 - val_loss: 0.1722 - val_accuracy: 0.9196

Epoch 00029: val_accuracy did not improve from 0.96429
Epoch 30/50
30/30 [==============================] - 9s 280ms/step - loss: 0.0481 - accuracy: 0.9893 - val_loss: 0.1951 - val_accuracy: 0.9107

Epoch 00030: val_accuracy did not improve from 0.96429
Epoch 31/50
30/30 [==============================] - 9s 285ms/step - loss: 0.0345 - accuracy: 0.9979 - val_loss: 0.1720 - val_accuracy: 0.9375

Epoch 00031: val_accuracy did not improve from 0.96429
Epoch 32/50
30/30 [==============================] - 9s 286ms/step - loss: 0.0466 - accuracy: 0.9893 - val_loss: 0.2559 - val_accuracy: 0.9018

Epoch 00032: val_accuracy did not improve from 0.96429
Epoch 33/50
30/30 [==============================] - 9s 291ms/step - loss: 0.0455 - accuracy: 0.9893 - val_loss: 0.1817 - val_accuracy: 0.9286

Epoch 00033: val_accuracy did not improve from 0.96429
Epoch

In [19]:
#Salva o modelo (pesos + conexões entre os neurônios, ou seja, a estrutura da rede)
model.save('C:\\Users\\LDT\\Desktop\\mestrado-unifesp\\exemplos\\modelo_budioes_resnet_' + optimizer_name + '_' + str(learning_rate) + ".model");

INFO:tensorflow:Assets written to: C:\Users\LDT\Desktop\mestrado-unifesp\exemplos\modelo_budioes_resnet_SGD_0.01.model\assets


C:\Users\LDT\anaconda3\envs\keras-env\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [20]:
#https://www.learndatasci.com/tutorials/hands-on-transfer-learning-keras/
#https://neptune.ai/blog/keras-metrics